In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb_top_1000.csv


In [5]:
!pip install -U sentence-transformers
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.0/451.0 kB 5.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.3 MB/s eta 0:00:00


In [67]:
from joblib import Parallel, delayed
from transformers import BertTokenizer, BertModel
from transformers import AlbertTokenizer, AlbertModel
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import numpy as np
import random
import string

# from transformers import AutoModel, AutoTokenizer



In [69]:
df = pd.read_csv("D:\Education\Marksmaster\semantic_search-main\imdb_top_1000.csv")
df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000"
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN


In [70]:
# add_series_title=" Movie name is "
# add_stars=" Stars are "
# add_director=" Director's name is "

In [71]:
overview_new=[]
for overvw,title,star1,star2,star3,star4,director in zip(df['Overview'],df['Series_Title'],df['Star1'],df['Star2'],df['Star3'],df['Star4'],df['Director']):
    
    add_series_title=" Movie name is "
    add_stars=" Stars are "
    add_director=" Director's name is "
    
    
    add_series_title+=title
    add_series_title+=" . "
    add_stars+=star1+" , "
    add_stars+=star2+" , "
    add_stars+=star3+" , "
    add_stars+=star4
    add_stars+=" ."
    add_director+=director+" . "
    overvw+=add_series_title
    overvw+=add_stars
    overvw+=add_director
    overview_new.append(overvw)
#     print(overvw)
#     print()

df['Overview_new']=overview_new
df
    
    
    
    

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Overview_new
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Two imprisoned men bond over a number of years...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",An organized crime dynasty's aging patriarch t...
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",When the menace known as the Joker wreaks havo...
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",The early life and career of Vito Corleone in ...
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",A jury holdout attempts to prevent a miscarria...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN,A young New York socialite becomes interested ...
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN,Sprawling epic covering the life of a Texas ca...
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000","In Hawaii in 1941, a private is cruelly punish..."
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN,Several survivors of a torpedoed merchant ship...


In [72]:

# Define the set of characters to use for generating random values
characters = string.ascii_letters + string.digits

# Initialize an empty set to store unique values
unique_values = set()

# Generate and add unique values until reaching the desired count
while len(unique_values) < 1000:
    random_value = ''.join(random.choice(characters) for _ in range(10))  # Adjust the length as needed
    unique_values.add(random_value)

# Convert the set to a list if needed
unique_values_list = list(unique_values)

# Print the first few unique values as an example
df['MovieID']=unique_values_list

In [73]:
df.sample(frac=1)
df.reset_index(drop=True)
new_df=df.iloc[:100,:]

In [74]:
new_df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Overview_new,MovieID
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Two imprisoned men bond over a number of years...,5vEqq1ytMd
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",An organized crime dynasty's aging patriarch t...,AGyNMbstEk
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",When the menace known as the Joker wreaks havo...,nZL4UvtITb
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",The early life and career of Vito Corleone in ...,9eIvXGxwqL
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",A jury holdout attempts to prevent a miscarria...,s8WBDi2Y3X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://m.media-amazon.com/images/M/MV5BNDg4Nj...,Amélie,2001,U,122 min,"Comedy, Romance",8.3,Amélie is an innocent and naive girl in Paris ...,69.0,Jean-Pierre Jeunet,Audrey Tautou,Mathieu Kassovitz,Rufus,Lorella Cravotta,703810,"33,225,499",Amélie is an innocent and naive girl in Paris ...,OQKrFyeOkG
96,https://m.media-amazon.com/images/M/MV5BMTA2ND...,Snatch,2000,UA,104 min,"Comedy, Crime",8.3,"Unscrupulous boxing promoters, violent bookmak...",55.0,Guy Ritchie,Jason Statham,Brad Pitt,Benicio Del Toro,Dennis Farina,782001,"30,328,156","Unscrupulous boxing promoters, violent bookmak...",25wkgrlQcw
97,https://m.media-amazon.com/images/M/MV5BOTdiNz...,Requiem for a Dream,2000,A,102 min,Drama,8.3,The drug-induced utopias of four Coney Island ...,68.0,Darren Aronofsky,Ellen Burstyn,Jared Leto,Jennifer Connelly,Marlon Wayans,766870,"3,635,482",The drug-induced utopias of four Coney Island ...,r6TbRbrSjG
98,https://m.media-amazon.com/images/M/MV5BNTBmZW...,American Beauty,1999,UA,122 min,Drama,8.3,A sexually frustrated suburban father has a mi...,84.0,Sam Mendes,Kevin Spacey,Annette Bening,Thora Birch,Wes Bentley,1069738,"130,096,601",A sexually frustrated suburban father has a mi...,ntmtQ3zNiX


In [75]:
new_df.drop(['Poster_Link', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Gross'],axis=1,inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_13136\2526846985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(['Poster_Link', 'Released_Year', 'Certificate',


In [76]:
## Assign random alphanumeric id to each movie



In [132]:
es=Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","1121@Elasticsearch"),
    ca_certs="C:/elasticsearch-8.13.0/config/certs/http_ca.crt"
)

es.ping()

True

In [78]:
new_df['Overview_new'].isnull().sum()

0

In [79]:
new_df['Overview_new'].value_counts()

Overview_new
Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency. Movie name is The Shawshank Redemption .  Stars are Tim Robbins , Morgan Freeman , Bob Gunton , William Sadler . Director's name is Frank Darabont .                                                                                                           1
Eight years after the Joker's reign of anarchy, Batman, with the help of the enigmatic Catwoman, is forced from his exile to save Gotham City from the brutal guerrilla terrorist Bane. Movie name is The Dark Knight Rises .  Stars are Christian Bale , Tom Hardy , Anne Hathaway , Gary Oldman . Director's name is Christopher Nolan .                                            1
A family heads to an isolated hotel for the winter where a sinister presence influences the father into violence, while his psychic son sees horrific forebodings from both past and future. Movie name is The Shining .  Stars are Jack Ni

In [80]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [81]:
new_df["Overview_newVector"] = new_df["Overview_new"].apply(lambda x: model.encode(x))


C:\Users\DELL\AppData\Local\Temp\ipykernel_13136\3901537213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Overview_newVector"] = new_df["Overview_new"].apply(lambda x: model.encode(x))


In [82]:
new_df

,Series_Title,Overview,Director,Star1,Star2,Star3,Star4,Overview_new,MovieID,Overview_newVector
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,Two imprisoned men bond over a number of years...,5vEqq1ytMd,"[0.050315782, 0.050354213, -0.007703683, 0.006..."
1,The Godfather,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,An organized crime dynasty's aging patriarch t...,AGyNMbstEk,"[0.01254484, 0.07579498, 0.003700674, 0.058907..."
2,The Dark Knight,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,When the menace known as the Joker wreaks havo...,nZL4UvtITb,"[0.050855033, 0.018460661, -0.010824432, -0.03..."
3,The Godfather: Part II,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,The early life and career of Vito Corleone in ...,9eIvXGxwqL,"[-0.0065082093, 0.104941905, 0.0018201503, 0.0..."
4,12 Angry Men,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,A jury holdout attempts to prevent a miscarria...,s8WBDi2Y3X,"[0.044500545, 0.023473129, 0.01756305, 0.02409..."
...,...,...,...,...,...,...,...,...,...,...
95,Amélie,Amélie is an innocent and naive girl in Paris ...,Jean-Pierre Jeunet,Audrey Tautou,Mathieu Kassovitz,Rufus,Lorella Cravotta,Amélie is an innocent and naive girl in Paris ...,OQKrFyeOkG,"[0.057287678, 0.054205652, 0.0005300591, 0.032..."
96,Snatch,"Unscrupulous boxing promoters, violent bookmak...",Guy Ritchie,Jason Statham,Brad Pitt,Benicio Del Toro,Dennis Farina,"Unscrupulous boxing promoters, violent bookmak...",25wkgrlQcw,"[0.05131458, 0.06742282, 0.0022847555, 0.06346..."
97,Requiem for a Dream,The drug-induced utopias of four Coney Island ...,Darren Aronofsky,Ellen Burstyn,Jared Leto,Jennifer Connelly,Marlon Wayans,The drug-induced utopias of four Coney Island ...,r6TbRbrSjG,"[0.029318776, 0.089185864, -0.005104835, -0.01..."
98,American Beauty,A sexually frustrated suburban father has a mi...,Sam Mendes,Kevin Spacey,Annette Bening,Thora Birch,Wes Bentley,A sexually frustrated suburban father has a mi...,ntmtQ3zNiX,"[0.030750565, 0.07373068, -0.00046243193, -0.0..."


In [83]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Series_Title        100 non-null    object
 1   Overview            100 non-null    object
 2   Director            100 non-null    object
 3   Star1               100 non-null    object
 4   Star2               100 non-null    object
 5   Star3               100 non-null    object
 6   Star4               100 non-null    object
 7   Overview_new        100 non-null    object
 8   MovieID             100 non-null    object
 9   Overview_newVector  100 non-null    object
dtypes: object(10)
memory usage: 7.9+ KB


In [84]:
new_df['Overview_newVector'][0].shape

(768,)

In [85]:
es.ping()

True

In [86]:
import os
os.getcwd()

'd:\\Education\\Marksmaster\\semantic_search-main'

In [116]:
from indexMapping import indexMapping
if es.indices.exists(index="all_moviess"):
    # Delete the existing index
    es.indices.delete(index="all_moviess")
es.indices.create(index="all_moviess", mappings=indexMapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_moviess'})

In [123]:
record_list = new_df.to_dict("records")


In [126]:
new_df

,Series_Title,Overview,Director,Star1,Star2,Star3,Star4,Overview_new,MovieID,Overview_newVector
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,Two imprisoned men bond over a number of years...,5vEqq1ytMd,"[0.050315782, 0.050354213, -0.007703683, 0.006..."
1,The Godfather,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,An organized crime dynasty's aging patriarch t...,AGyNMbstEk,"[0.01254484, 0.07579498, 0.003700674, 0.058907..."
2,The Dark Knight,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,When the menace known as the Joker wreaks havo...,nZL4UvtITb,"[0.050855033, 0.018460661, -0.010824432, -0.03..."
3,The Godfather: Part II,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,The early life and career of Vito Corleone in ...,9eIvXGxwqL,"[-0.0065082093, 0.104941905, 0.0018201503, 0.0..."
4,12 Angry Men,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,A jury holdout attempts to prevent a miscarria...,s8WBDi2Y3X,"[0.044500545, 0.023473129, 0.01756305, 0.02409..."
...,...,...,...,...,...,...,...,...,...,...
95,Amélie,Amélie is an innocent and naive girl in Paris ...,Jean-Pierre Jeunet,Audrey Tautou,Mathieu Kassovitz,Rufus,Lorella Cravotta,Amélie is an innocent and naive girl in Paris ...,OQKrFyeOkG,"[0.057287678, 0.054205652, 0.0005300591, 0.032..."
96,Snatch,"Unscrupulous boxing promoters, violent bookmak...",Guy Ritchie,Jason Statham,Brad Pitt,Benicio Del Toro,Dennis Farina,"Unscrupulous boxing promoters, violent bookmak...",25wkgrlQcw,"[0.05131458, 0.06742282, 0.0022847555, 0.06346..."
97,Requiem for a Dream,The drug-induced utopias of four Coney Island ...,Darren Aronofsky,Ellen Burstyn,Jared Leto,Jennifer Connelly,Marlon Wayans,The drug-induced utopias of four Coney Island ...,r6TbRbrSjG,"[0.029318776, 0.089185864, -0.005104835, -0.01..."
98,American Beauty,A sexually frustrated suburban father has a mi...,Sam Mendes,Kevin Spacey,Annette Bening,Thora Birch,Wes Bentley,A sexually frustrated suburban father has a mi...,ntmtQ3zNiX,"[0.030750565, 0.07373068, -0.00046243193, -0.0..."


In [125]:
len(record_list)

100

In [119]:
for record in record_list:
    try:
        es.index(index="all_moviess", document=record, id=record["MovieID"])
    except Exception as e:
        print(e)

In [120]:
es.count(index="all_moviess")


ObjectApiResponse({'count': 100, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [129]:
input_keyword = "Good Jagten"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "Overview_newVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 3,
    "num_candidates" : 100, 
}

res = es.knn_search(index="all_moviess", knn=query , source=["Series_Title","Overview"])
res["hits"]["hits"]

C:\Users\DELL\AppData\Local\Temp\ipykernel_13136\186936335.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_moviess", knn=query , source=["Series_Title","Overview"])


[{'_index': 'all_moviess',
  '_id': 'pHYPHHeMiF',
  '_score': 0.3942695,
  '_source': {'Series_Title': 'Jagten',
   'Overview': "A teacher lives a lonely life, all the while struggling over his son's custody. His life slowly gets better as he finds love and receives good news from his son, but his new luck is about to be brutally shattered by an innocent little lie."}},
 {'_index': 'all_moviess',
  '_id': '5j9UO9QWSn',
  '_score': 0.36175203,
  '_source': {'Series_Title': 'The Great Dictator',
   'Overview': "Dictator Adenoid Hynkel tries to expand his empire while a poor Jewish barber tries to avoid persecution from Hynkel's regime."}},
 {'_index': 'all_moviess',
  '_id': '3UGd41Ejb5',
  '_score': 0.36050344,
  '_source': {'Series_Title': "Schindler's List",
   'Overview': 'In German-occupied Poland during World War II, industrialist Oskar Schindler gradually becomes concerned for his Jewish workforce after witnessing their persecution by the Nazis.'}}]

In [109]:
def read_yaml(path_to_yaml:str) -> dict:
    with open(path_to_yaml) as yaml_file:
        content = yaml.safe_load(yaml_file)

        return content


In [110]:
n=read_yaml("./constants.yaml")
num_candidates=n['no_of_rows_to_be_used']
no_outputs_to_show=n['no_outputs_to_show']

In [113]:
type(num_candidates)

int

In [114]:
num_candidates

100

In [22]:
# new_df.to_csv("400_semantic.csv",index=False)

In [23]:
# tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
# model = BertModel.from_pretrained('bert-large-cased')


In [24]:





# def embed_sentence(text):
#     inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     embeddings = torch.mean(outputs.last_hidden_state, dim=1)
#     return embeddings.numpy()


In [25]:
# embeddings_list = [embed_sentence(text) for text in new_df['Overview_new']]


In [26]:
# user_input = input("Enter a sentence: ")

# # Embed user input
# user_embedding = embed_sentence(user_input)

# # Reshape embeddings to remove batch dimension
# user_embedding = user_embedding.squeeze(0)  # Remove batch dimension
# embeddings_list = [embedding.squeeze(0) for embedding in embeddings_list]

# # Search vector database using cosine similarity
# closest_entry = None
# max_similarity = -1  # Initialize max similarity to a minimum value

# for idx, embedding in enumerate(embeddings_list):
#     similarity = cosine_similarity([user_embedding], [embedding])[0][0]
#     if similarity > max_similarity:
#         max_similarity = similarity
#         closest_entry = new_df['Overview_new'].iloc[idx]

# # Display closest entry
# print("Closest entry:", closest_entry)

In [30]:
# Download and load the pre-trained BERT model from Hugging Face
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a sentence embedding model
sentence_transformer_model = SentenceTransformer(model_name)

In [31]:
from elasticsearch import Elasticsearch

# Initialize Elasticsearch client for local setup
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])



# Initialize Elasticsearch client
# es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# Function to embed and store the sentence in Elasticsearch
def embed_and_store_sentence(sentence, index_name):
    # Tokenize and encode the sentence using BERT
    tokens = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**tokens)
        embedding = torch.mean(outputs.last_hidden_state, dim=1).squeeze().numpy()
    # Store the embedding in Elasticsearch
    es.index(index=index_name, body={"sentence": sentence, "embedding": embedding.tolist()})

# Function to search for similar vectors and display results
def search_and_display_similar_vectors(user_input, index_name):
    # Embed the user input
    user_embedding = sentence_transformer_model.encode(user_input)
    # Search Elasticsearch for similar vectors
    search_query = {
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {"query_vector": user_embedding.tolist()},
                },
            }
        }
    }
    response = es.search(index=index_name, body=search_query)
    # Display top two similar vectors
    for hit in response["hits"]["hits"][:2]:
        similarity = hit["_score"]
        similar_vector = np.array(hit["_source"]["embedding"])
        print("Similar Vector:", similar_vector)
        print("Similarity Index:", similarity)

# Example usage
new_df = ...  # Your DataFrame with the column "Overview_new"
index_name = "sentence_embeddings"

# Embed and store sentences from DataFrame in Elasticsearch
for sentence in new_df["Overview_new"]:
    embed_and_store_sentence(sentence, index_name)

# Take input from the user
user_input = input("Enter a sentence: ")

# Search and display similar vectors
search_and_display_similar_vectors(user_input, index_name)


TypeError: __init__() missing 1 required positional argument: 'scheme'

In [101]:
read_yaml("./constants.yaml")

{'no_of_rows_to_be_used': 100}